In [1]:
!pip freeze

chardet==4.0.0
idna==2.10
kafka==1.3.5
kafka-python==2.0.2
numpy==1.21.0
requests==2.25.1
urllib3==1.26.6


In [2]:
import requests
import json
import random
import os

In [3]:
#HELPER FUNCTIONS FOR PINGING SERVICES RUNNING ON PORT 8083
#FUNCTIONS USE ALL HTTP METHODS AND GENERATE TRACES
def ratings_get_req(ID=0, by_user=False, by_movie=False, by_rating_id=False):
    
    #check to make sure only one search option is selected
    if((by_user and by_movie and by_rating_id)or 
       (by_user and by_movie)or 
       (by_movie and by_rating_id)or 
       (by_user and by_rating_id)):
        print("ERROR: Search options are mutually exclusive.")
        return
    
    req_string = 'http://localhost:8083/ratings'
    
    if(by_rating_id): #check arguments for get by rating id
        req_string = req_string+ f'/{ID}'
    if(by_user): #check arguments for get by user
        req_string = req_string+ f'/users/{ID}'
    if(by_movie): #check arguments for get by movie
        req_string = req_string+ f'/movies/{ID}'
    
    req = requests.get(req_string)
    print(f'call to: {req_string}')
    
def ratings_put_req(ID, movie_id, user_id, rating):
    req_string = f'http://localhost:8083/ratings/{ID}'
    req = requests.put(req_string, 
                       json={"movie_id": movie_id, 
                              "user_id": user_id, 
                              "rating": rating})
    print(f'call to: {req_string}')
    
def ratings_del_req(ID):
    req_string = f'http://localhost:8083/ratings/{ID}'
    req = requests.delete(req_string)
    print(f'call to: {req_string}')
    
def ratings_post_req(movie_id, user_id, rating):
    req_string = 'http://localhost:8083/ratings'
    req = requests.post(req_string, 
                        json={"movie_id": movie_id, 
                              "user_id": user_id, 
                              "rating": rating})
    print(f'call to: {req_string}')
    return req.json()['id']

In [4]:
#HELPER FUNCTIONS FOR PINGING SERVICE RUNNING ON PORT 8081
#FUNCTIONS WILL GENERATE LOGS FOR ALL HTTP METHODS
def movies_get_req(ID=-1):
    if(ID < 0):
        req_string = 'http://localhost:8081/movies'
    else:
        req_string = f'http://localhost:8081/movies/{ID}'
    req = requests.get(req_string)
    print(f'call to: {req_string}')
    
def movies_post_req(name, description):
    req_string = 'http://localhost:8081/movies'
    req = requests.post(req_string, 
                        json={"name": name, 
                              "description": description})
    print(f'call to: {req_string}')
    return req.json()['id']
    
def movies_put_req(ID, name, description):
    req_string = f'http://localhost:8081/movies/{ID}'
    req = requests.post(req_string, 
                        json={"name": name, 
                              "description": description})
    print(f'call to: {req_string}')
    
def movies_del_req(ID):
    req_string = f'http://localhost:8081/movies/{ID}'
    req = requests.delete(req_string)
    print(f'call to: {req_string}')

In [5]:
#FUNCTIONS PINGS THE CATALOG SERVICE AND REQUIRES A USERID
def catalog_get_req(ID):
    req_string = f'http://localhost:8082/catalog/{ID}'
    req = requests.get(req_string)
    print(f'call to: {req_string}')

In [6]:
def gen_logs(count):
    count = count//2 #halve the count because we do this process twice
    get_count = count//2 #get reqs are more common on web servers
    put_count = get_count//2 #put requests will not recreate resources so we want more of these
    post_count = put_count//2 #post requests may have unwanted side affects
    del_count = post_count//2 #delete must be less than the post and put counts
    
    random.seed()
    
    movie_ids = []
    rating_ids = []
    
    for i in range (get_count):
        
        #variables for ratings get request
        r = random.choice([1, 2, 3, 0])
        by_user = True if r == 1 else False
        by_movie = True if r == 2 else False
        by_rating_id = True if r == 3 else False
        
        ratings_get_req(random.randint(0,30), by_user, by_movie, by_rating_id)
        movies_get_req(random.randint(0,30))
        catalog_get_req(random.randint(0,30))
        
    for i in range (put_count):
        movie_id = random.randint(0,12)
        user_id = random.randint(0,12)
        rating = random.randint(1,8)
        
        ratings_put_req(random.randint(0,30), movie_id, user_id, rating)        
        movies_put_req(random.randint(0,30), 'test', 'test')
   
    for i in range (post_count):
        movie_id = random.randint(0,12)
        user_id = random.randint(0,12)
        rating = random.randint(1,8)
        
        r_id = ratings_post_req(movie_id, user_id, rating)
        m_id = movies_post_req(f'test{i}', f'test{i}')
        
        rating_ids.append(r_id)
        movie_ids.append(m_id)
        
    for i in range (del_count):
        r_del = random.choice(rating_ids)
        m_del = random.choice(movie_ids)
        ratings_del_req(r_del)
        movies_del_req(m_del)
        rating_ids.remove(r_del)
        movie_ids.remove(m_del)
        
    print('finished...')

gen_logs(1000)

call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/21
call to: http://localhost:8082/catalog/13
call to: http://localhost:8083/ratings/28
call to: http://localhost:8081/movies/17
call to: http://localhost:8082/catalog/15
call to: http://localhost:8083/ratings/users/2
call to: http://localhost:8081/movies/14
call to: http://localhost:8082/catalog/8
call to: http://localhost:8083/ratings/users/26
call to: http://localhost:8081/movies/23
call to: http://localhost:8082/catalog/26
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/0
call to: http://localhost:8082/catalog/9
call to: http://localhost:8083/ratings/users/14
call to: http://localhost:8081/movies/5
call to: http://localhost:8082/catalog/29
call to: http://localhost:8083/ratings/users/7
call to: http://localhost:8081/movies/15
call to: http://localhost:8082/catalog/4
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/15
call to: http://localhost:8082/cat

call to: http://localhost:8082/catalog/2
call to: http://localhost:8083/ratings/movies/10
call to: http://localhost:8081/movies/29
call to: http://localhost:8082/catalog/29
call to: http://localhost:8083/ratings/movies/11
call to: http://localhost:8081/movies/0
call to: http://localhost:8082/catalog/7
call to: http://localhost:8083/ratings/29
call to: http://localhost:8081/movies/9
call to: http://localhost:8082/catalog/4
call to: http://localhost:8083/ratings/16
call to: http://localhost:8081/movies/25
call to: http://localhost:8082/catalog/28
call to: http://localhost:8083/ratings/9
call to: http://localhost:8081/movies/6
call to: http://localhost:8082/catalog/13
call to: http://localhost:8083/ratings/users/14
call to: http://localhost:8081/movies/14
call to: http://localhost:8082/catalog/18
call to: http://localhost:8083/ratings/movies/29
call to: http://localhost:8081/movies/20
call to: http://localhost:8082/catalog/12
call to: http://localhost:8083/ratings
call to: http://localhos

call to: http://localhost:8082/catalog/10
call to: http://localhost:8083/ratings/18
call to: http://localhost:8081/movies/29
call to: http://localhost:8082/catalog/25
call to: http://localhost:8083/ratings/movies/2
call to: http://localhost:8081/movies/8
call to: http://localhost:8082/catalog/11
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/20
call to: http://localhost:8082/catalog/17
call to: http://localhost:8083/ratings/users/5
call to: http://localhost:8081/movies/2
call to: http://localhost:8082/catalog/14
call to: http://localhost:8083/ratings/users/22
call to: http://localhost:8081/movies/3
call to: http://localhost:8082/catalog/20
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/28
call to: http://localhost:8082/catalog/7
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/1
call to: http://localhost:8082/catalog/11
call to: http://localhost:8083/ratings/users/16
call to: http://localhost:8081/m

call to: http://localhost:8082/catalog/27
call to: http://localhost:8083/ratings/users/2
call to: http://localhost:8081/movies/26
call to: http://localhost:8082/catalog/23
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/25
call to: http://localhost:8082/catalog/24
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/24
call to: http://localhost:8082/catalog/24
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/28
call to: http://localhost:8082/catalog/21
call to: http://localhost:8083/ratings/users/6
call to: http://localhost:8081/movies/27
call to: http://localhost:8082/catalog/11
call to: http://localhost:8083/ratings/movies/20
call to: http://localhost:8081/movies/25
call to: http://localhost:8082/catalog/23
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies/4
call to: http://localhost:8082/catalog/7
call to: http://localhost:8083/ratings/movies/14
call to: http://localhost:8081/

call to: http://localhost:8081/movies/16
call to: http://localhost:8083/ratings/17
call to: http://localhost:8081/movies/19
call to: http://localhost:8083/ratings/7
call to: http://localhost:8081/movies/3
call to: http://localhost:8083/ratings/16
call to: http://localhost:8081/movies/30
call to: http://localhost:8083/ratings/2
call to: http://localhost:8081/movies/22
call to: http://localhost:8083/ratings/0
call to: http://localhost:8081/movies/4
call to: http://localhost:8083/ratings/3
call to: http://localhost:8081/movies/27
call to: http://localhost:8083/ratings/17
call to: http://localhost:8081/movies/22
call to: http://localhost:8083/ratings/29
call to: http://localhost:8081/movies/21
call to: http://localhost:8083/ratings/18
call to: http://localhost:8081/movies/0
call to: http://localhost:8083/ratings/3
call to: http://localhost:8081/movies/15
call to: http://localhost:8083/ratings/5
call to: http://localhost:8081/movies/16
call to: http://localhost:8083/ratings/29
call to: http

call to: http://localhost:8083/ratings/5
call to: http://localhost:8081/movies/18
call to: http://localhost:8083/ratings/9
call to: http://localhost:8081/movies/23
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8081/movies
call to: http://localhost:8083/ratings
call to: http://localhost:8